In [108]:
import numpy as np
#from imageio import imread, imwrite
import cv2
# from matplotlib,pyplot as plt # not working for me

## Naive Method: Increase blue channel

In [109]:
image = cv2.imread('grainger.jpg')

arr = image * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
cv2.imwrite("night_final.png", gamma_img)

True

## Advanced Method: Remove illumination, decrease exposure, relight with night illuminants, and add noise

### Remove illumination

Stacked RGGB Bayer image:
$$ \mathbf{I}_{day} \in R^{\frac{H}{2} \times \frac{W}{2} \times 4} $$

In [110]:
img = cv2.imread('grainger.jpg')
n_channels = 4
I_day = np.zeros((img.shape[0], img.shape[1], n_channels))
I_day[:,:,0] = img[:,:,0]
I_day[:,:,1] = img[:,:,1]
I_day[:,:,2] = img[:,:,1].copy()
I_day[:,:,3] = img[:,:,2]
print(I_day.shape)
print(I_day)


(1200, 1800, 4)
[[[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 [[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 [[240. 226. 226. 220.]
  [240. 226. 226. 220.]
  [240. 226. 226. 220.]
  ...
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]
  [246. 246. 246. 246.]]

 ...

 [[ 51.  98.  98.  89.]
  [ 50.  97.  97.  88.]
  [ 34.  81.  81.  72.]
  ...
  [ 50.  96.  96.  83.]
  [ 40.  88.  88.  76.]
  [ 24.  73.  73.  59.]]

 [[ 49.  96.  96.  87.]
  [ 31.  78.  78.  69.]
  [ 35.  82.  82.  73.]
  ...
  [ 38.  85.  85.  69.]
  [ 34.  83.  83.  69.]
  [ 31.  80.  80.  64.]]

 [[ 52.  99.  99.  90.]
  [ 30.  77.  77.  68.]
  [ 49.  96.  96.  87.]
  ...
  [ 28.  75.  75.  59.]
  [ 28.  75.  75.  59.]
  [ 37.  86.  86.  70.]]]


In [111]:
# The process begins with a "minimally processed Bayer image recorded by the camera sensor".
# We could synthesize the effect of a raw Bayer image with the code below,
# then interpolate the pixel values (or downsample, but I suppose it's arbitrary). This is why I think their I_day is H/2 x W/2
# OR this may all be useless and we should just contain with the normal rgb image turned into rggb

# Created with the help of ChatGPT.
# def apply_bayer_pattern(image):
#     print(image.shape)
#     # Create a blank Bayer pattern image with the same dimensions as the RGB image
#     # bayer_image = np.zeros(image.shape * 2)

#     new_height, new_width = image.shape[0] * 2, image.shape[1] * 2
#     bayer = np.zeros((new_height, new_width))
#     bayer[0::2, 0::2] = image[:, :, 0]
#     bayer[0::2, 1::2] = image[:, :, 1]
#     bayer[1::2, 0::2] = image[:, :, 1]
#     bayer[1::2, 1::2] = image[:, :, 2]

#     return bayer

# I_day = apply_bayer_pattern(img)
# cv2.imwrite('bayer_image.jpg', I_day)


Normalized image:
$$ \mathbf{I}_n = (\mathbf{I}_{day}-b_l) / (w_l-b_l) $$

In [112]:
# wl = np.mean(np.max(img, axis=(0,1)))
# bl = np.mean(np.min(img, axis=(0,1)))
wl = np.max(I_day)
bl = np.min(I_day)

I_n = (I_day - bl) / (wl - bl)
cv2.imwrite('I_n.jpg', I_n * 255)


True

White-balanced image:
$$ \mathbf{I}_w = \mathbf{I}_n\mathbf{L}_{day} $$
where
$$ {L}_{day} = diag(\frac{1}{b}, \frac{1}{g}, \frac{1}{r}) $$

In [113]:
L_day = np.identity(I_n.shape[-1])
I_n_avg = np.mean(I_n, axis=(0, 1))
for i in range(L_day.shape[1]):
    L_day[i, i] = 1 / I_n_avg[i]

I_w = I_n @ L_day
cv2.imwrite('I_w.jpg', I_w * 255)

# We might just have to come up with some values for r, g, b to be the average shift of intensities to color balance to neutral
# Or create a simple algorithm

True

### Lowering brightness

`D`: dictionary for the normalized mean intensity value of each Bayer image
(could just make list with index as key for the image)

In [114]:
# I believe bayer_images should have a few nighttime images, which would make D have normalized means of random nighttime photos
bayer_images = np.array([I_day])
print(bayer_images.shape)

# contains normalized mean intensity values of each Bayer image
D = np.array([np.mean(img) / np.max(img) for img in bayer_images])

global_scale_factor_d = np.random.choice(D)
print(global_scale_factor_d)

I_e = I_w * global_scale_factor_d


(1, 1200, 1800, 4)
0.5865332874909223


### Adding illuminants

`L`: dictionary for nighttime illuminants

In [115]:
# Make a few samples of nighttime illuminants

# Number of nighttime illuminants
M = 5

# Create 5 dummy illuminants
illuminants = np.random.rand(M * np.prod(I_e.shape)).reshape((M,) + I_e.shape)

chromaticity_pairs = []
for illum in illuminants:
    # Joint chromaticity values, normalized
    rg = illum[:,:,0] / illum[:,:,1]
    rg = (rg - np.min(rg)) / np.max(rg)
    gb = illum[:,:,-2] / illum[:,:,-1]
    gb = (gb - np.min(gb)) / np.max(gb)

    # chromaticity_pairs.append((rg, gb))
    chromaticity_pairs.append((np.mean(rg), np.mean(gb)))

chromaticity_pairs = np.array(chromaticity_pairs)
print(chromaticity_pairs.shape)

# Mean of normalized chromaticity values in L
# mu = np.mean(chromaticity_pairs, axis=(0, 2, 3))
mu = np.mean(chromaticity_pairs, axis=0)
print(mu, mu.shape)

# Covariance of normalized chromaticity values in L
# sigma = np.cov()
sigma = np.zeros((mu.shape[0], mu.shape[0]))
for pair in chromaticity_pairs:
    sigma += np.outer((pair - mu).T, pair - mu)
sigma /= M
print(sigma, sigma.shape)

(5, 2)
[4.23796389e-06 9.79316087e-06] (2,)
[[9.67166836e-12 2.08681963e-12]
 [2.08681963e-12 5.85150446e-11]] (2, 2)


### Adding noise

### Final Result